In [1]:
from dataclasses import dataclass

In [2]:
with open("input.txt", "rt") as f:
    platform = f.read().strip().split("\n")

# Part 1

In [3]:
platform_t = list(map(list, zip(*platform)))
n_rows = len(platform)

# Roll north
for col in platform_t:
    cursor = 1
    while cursor < n_rows:
        if cursor > 0 and col[cursor] == "O" and col[cursor - 1] == ".":
            col[cursor], col[cursor - 1] = ".", "O"
            cursor -= 1
        else:
            cursor += 1

# Calculate total load
load = 0
for col in platform_t:
    for y, cell in enumerate(col):
        if cell == "O":
            load += n_rows - y 
load

113078

# Part 2

The idea is that at some point there will be a cycle of rock positions, so if we can reach that cycle, we can calculate load after nth platform spin cycle. I also assume that the length of a cycle of rock positions is much smaller than 1_000_000_000 and that it starts early.

In [4]:
@dataclass
class Cell:
    value: str

# Using cell objects allows to create "views", if I change a value of a cell
# in one view, it also is changes in the other one (makes spinning the platform easier)
platform_view = [[Cell(value) for value in row] for row in platform]
platform_view_t = list(map(list, zip(*platform_view)))

loads = []
visited = []

while True:

    # Platform spin cycle
    
    for direction in ("n", "w", "s", "e"):
        view = platform_view if direction in ("w", "e") else platform_view_t

        for col in view:
            if direction in ("e", "s"):
                col = list(reversed(col))

            cursor = 1
            while cursor < n_rows:
                if cursor > 0 and col[cursor].value == "O" and col[cursor - 1].value == ".":
                    col[cursor].value, col[cursor - 1].value = ".", "O"
                    cursor -= 1
                else:
                    cursor += 1

    # Calcualte and store load

    load = 0
    for col in platform_view_t:
        for y, cell in enumerate(col):
            if cell.value == "O":
                load += n_rows - y
    loads.append(load)

    # Check if rocks were in this position previously (we hit a cycle of rock positions)

    platform_str = "".join(cell.value for row in platform_view for cell in row)
    if platform_str in visited:
        cycle_start = visited.index(platform_str)
        cycle_length = len(visited) - cycle_start
        break
    visited.append(platform_str)

# Lookup load after bilion spin cycles
load_idx = (1_000_000_000 - cycle_start) % cycle_length + cycle_start - 1
loads[load_idx]

94255